In [1]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold


In [3]:
# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)
train_set = data.build_full_trainset()
train_set

In [10]:
# ALS优化
# bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
# bsl_options = {'method': 'sgd','n_epochs': 5}
# algo = BaselineOnly(bsl_options=bsl_options)
#algo = BaselineOnly()
algo = NormalPredictor()

kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)
uid = str(196)
iid = str(302)

# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

RMSE: 1.4320
RMSE: 1.4304
RMSE: 1.4325
user: 196        item: 302        r_ui = 4.00   est = 3.51   {'was_impossible': False}


In [7]:
pred = algo.predict(uid, iid, verbose=True)

user: 196        item: 302        r_ui = None   est = 4.20   {'was_impossible': False}


In [8]:
algo.predict?

Signature: algo.predict(uid, iid, r_ui=None, clip=True, verbose=False)
Docstring:
Compute the rating prediction for given user and item.

The ``predict`` method converts raw ids to inner ids and then calls the
``estimate`` method which is defined in every derived class. If the
prediction is impossible (e.g. because the user and/or the item is
unkown), the prediction is set according to :meth:`default_prediction()
<surprise.prediction_algorithms.algo_base.AlgoBase.default_prediction>`.

Args:
    uid: (Raw) id of the user. See :ref:`this note<raw_inner_note>`.
    iid: (Raw) id of the item. See :ref:`this note<raw_inner_note>`.
    r_ui(float): The true rating :math:`r_{ui}`. Optional, default is
        ``None``.
    clip(bool): Whether to clip the estimation into the rating scale.
        For example, if :math:`\hat{r}_{ui}` is :math:`5.5` while the
        rating scale is :math:`[1, 5]`, then :math:`\hat{r}_{ui}` is
        set to :math:`5`. Same goes if :math:`\hat{r}_{ui} < 1`.
   